In [ ]:
!pip install torcheeg

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.1/161.1 KB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.5/298.5 KB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.6/41.6 KB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 60.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.8/230.8 KB 9.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 KB 48.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 KB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 85.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 KB 6.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
from rich.pretty import pprint
from torcheeg.datasets import DREAMERDataset
from torcheeg.datasets.constants.emotion_recognition.dreamer import DREAMER_CHANNEL_LOCATION_DICT
from torcheeg import transforms
from torch import nn
import torch
import random

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
dataset_path = "/content/drive/MyDrive/Neuroscience/DREAMER.mat"
base_path = "/content/drive/MyDrive/Neuroscience/Arousal1/"

# dataset_path = "/content/drive/MyDrive/Neuroscience/DREAMER.mat"
# base_path = "/content/drive/MyDrive/Neuroscience/"

In [ ]:
dataset = DREAMERDataset(
    io_path=base_path + 'dreamer8sec',
    mat_path=dataset_path,
    offline_transform=transforms.Compose([
        transforms.BaselineRemoval(),
        transforms.MeanStdNormalize(),
        transforms.To2d()
    ]),
    # online_transform=transforms.ToTensor(),
    label_transform=transforms.Compose(
        [transforms.Select('arousal'),
         transforms.Binary(3.0)]),
    chunk_size=976,
    baseline_chunk_size=976,
    num_baseline=8
)

The target folder already exists, if you need to regenerate the database IO, please delete the path /content/drive/MyDrive/Neuroscience/Arousal1/dreamer8sec.


In [ ]:
def get_tf_feature(eeg, sr, n_channels = 14):
    WinLength = int(0.5*sr) # 500 points (0.5 sec, 500 ms)
    step = int(0.025*sr) # 25 points (or 25 ms)
    final_features = None
    for i in range(n_channels):
        eeg_single = eeg[i].squeeze()
        myparams = dict(nperseg = WinLength, noverlap = WinLength-step, return_onesided=True, mode='magnitude')
        f, nseg, Sxx = signal.spectrogram(x = eeg_single, fs = sr, **myparams)
        if(isinstance(final_features, np.ndarray)):
            final_features = np.concatenate((final_features, Sxx), axis=0)
        else:
            final_features = Sxx
    return final_features

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
# device = "cpu"
print(f"Using {device} device")

Using cuda device


In [ ]:
def convert_data_to_tensor(data):
    data = data.astype("float32")
    data = data.reshape(1, data.shape[0], data.shape[1])
    return torch.from_numpy(data)

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv2D_1 = nn.Sequential(
            nn.Conv2d(1, 1024, 11, stride=3),
            nn.Conv2d(1024, 512, 7, stride=3),
            nn.Conv2d(512, 128, 7, stride=3),
        )
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            # nn.Linear(14550, 2048),
            # nn.ReLU(),
            nn.Linear(17280, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Linear(128, 2),
        )

    def forward(self, x):
        x = self.conv2D_1(x)
        # x = self.flatten(x)
        x = x.view(1, -1)
        # print(x.shape)
        logits = self.linear_relu_stack(x)
        return logits

In [ ]:
random.seed(42)

test_size = 2000
test_index = random.sample(range(0, 11000), test_size)
train_index = []

for i in range(11040):
    if i not in test_index:
        train_index.append(i)
random.shuffle(train_index)

In [ ]:
def train_loop(dataset,  model, loss_fn, optimizer):
    # size = len(dataset)
    model.train()
    sample_size = len(train_index)
    j=0
    correct = 0
    for i in train_index:
        # Compute prediction and loss
        # print(i)
        # Backpropagation
        optimizer.zero_grad()
        
        X, y = dataset[i][0][0], dataset[i][1]
        X = get_tf_feature(X, sr=128)
        X = convert_data_to_tensor(X)
        if y == 0:
            y = [0]
        else:
            y = [1]
        y = torch.tensor(y)
        X, y = X.to(device), y.to(device)
        pred = model(X)
        # print(y.shape, pred.shape)
        loss = loss_fn(pred, y)
        
        correct += (pred.argmax(1) == y).type(torch.float).sum().item()

        loss.backward()
        optimizer.step()
        if j % 100 == 0:
            loss, current = loss.item(), j + 1
            print(f"loss: {loss}  [{current:>5d}/{sample_size:>5d}]")
            # if j%1000 == 0:
              # torch.save(model.state_dict(), "eeg_model_1000.pth")
        j=j+1
    correct /= sample_size
    str = (f"Train Accuracy: {(100*correct):>0.1f}\n")
    print(str)
    with open(base_path + "log.txt", "a") as f:
        f.write(str)

In [ ]:

val_error = 999999999.9
best_model_parameter = None

def test_loop(dataset, model, loss_fn):
    global val_error
    global best_model_parameter
    # size = len(dataset)
    test_loss, correct = 0, 0
    sample_size = len(test_index)
    # l = random.sample(range(0, 11000), 1)
    # j=0
    with torch.no_grad():
        model.eval()
        for i in test_index:
            X, y = dataset[i][0][0], dataset[i][1]
            X = get_tf_feature(X, sr=128)
            X = convert_data_to_tensor(X)
            if y == 0:
                y = [0]
            else:
                y = [1]
            y = torch.tensor(y)
            
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= sample_size
    correct /= sample_size

    if val_error > test_loss:
        val_error = test_loss
        best_model_parameter = model.state_dict()

    str = (f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    print(str)
    with open(base_path + "log.txt", "a") as f:
        f.write(str)

In [ ]:
learning_rate = 3e-6

In [ ]:
last_epoch = 9
model = NeuralNetwork().to(device)
if last_epoch >= 0:
  model.load_state_dict(torch.load(base_path + f'eeg_model_{last_epoch}.pth', map_location=torch.device('cpu')))
model.eval()

NeuralNetwork(
  (conv2D_1): Sequential(
    (0): Conv2d(1, 1024, kernel_size=(11, 11), stride=(3, 3))
    (1): Conv2d(1024, 512, kernel_size=(7, 7), stride=(3, 3))
    (2): Conv2d(512, 128, kernel_size=(7, 7), stride=(3, 3))
  )
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=17280, out_features=1024, bias=True)
    (1): ReLU()
    (2): Linear(in_features=1024, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=128, bias=True)
    (5): ReLU()
    (6): Linear(in_features=128, out_features=2, bias=True)
  )
)

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

epochs = 100
for t in range(epochs):
    if t <= last_epoch:
      continue
    str = (f"Epoch {t+1}\n-------------------------------\n")
    print(str)
    with open(base_path + "log.txt", "a") as f:
        f.write(str)
    train_loop(dataset, model, loss_fn, optimizer)
    test_loop(dataset, model, loss_fn)
    torch.save(model.state_dict(), base_path + f"eeg_model_{t}.pth")
print("Done!")

Epoch 11
-------------------------------

loss: 4.768370445162873e-07  [    1/ 9040]
loss: 1.1920928244535389e-07  [  101/ 9040]
loss: 9.16677454370074e-05  [  201/ 9040]
loss: 0.003986389376223087  [  301/ 9040]
loss: 1.1920928244535389e-07  [  401/ 9040]
loss: 0.00010179955279454589  [  501/ 9040]
loss: 0.0  [  601/ 9040]
loss: 0.022461412474513054  [  701/ 9040]
loss: 0.014150167815387249  [  801/ 9040]
loss: 0.022915199398994446  [  901/ 9040]
loss: 0.002562693553045392  [ 1001/ 9040]
loss: 7.152555099310121e-07  [ 1101/ 9040]
loss: 0.053099375218153  [ 1201/ 9040]
loss: 0.04856870695948601  [ 1301/ 9040]
loss: 0.012437872588634491  [ 1401/ 9040]
loss: 0.03686638921499252  [ 1501/ 9040]
loss: 0.0012317459331825376  [ 1601/ 9040]
loss: 3.576278118089249e-07  [ 1701/ 9040]
loss: 0.004210892133414745  [ 1801/ 9040]
loss: 0.15342797338962555  [ 1901/ 9040]
loss: 0.09003865718841553  [ 2001/ 9040]
loss: 1.0251946150674485e-05  [ 2101/ 9040]
loss: 0.0  [ 2201/ 9040]
loss: 0.0002594849502

KeyboardInterrupt: ignored

In [ ]:
assert best_model_parameter is not None, "No best model"
best_model = NeuralNetwork().to(device)
best_model.load_state_dict(best_model_parameter)
torch.save(best_model.state_dict(), "eeg_model.pth")

In [ ]:
!cp ./log.txt ./drive/MyDrive/Neuroscience